In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import sqlalchemy as sq
import re
from tqdm import tqdm
import jieba

In [37]:
defaultList = []
cedictList = []

In [38]:
defaultdict = r"""C:\Users\jw\Anaconda3\Lib\site-packages\jieba\dict.txt.big.txt"""

In [39]:
defaultdict

'C:\\Users\\jw\\Anaconda3\\Lib\\site-packages\\jieba\\dict.txt.big.txt'

In [42]:
f = open(defaultdict, "r",encoding="utf8")
for x in f:
    defaultList.append(re.split(string=x,pattern="\s",maxsplit=1)[0])

In [45]:
cedict = r"""C:\Users\jw\Downloads\cedict_1_0_ts_utf-8_mdbg\cedict_ts.txt"""

In [46]:
f = open(cedict, "r",encoding="utf8")
for x in f:
    cedictList.append(re.split(string=x,pattern="\s",maxsplit=1)[0])

In [51]:
defaultList

['1号店',
 '1號店',
 '4S店',
 '4s店',
 'AA制',
 'AB型',
 'AT&T',
 'A型',
 'A座',
 'A股',
 'A輪',
 'A轮',
 'BB机',
 'BB機',
 'BP机',
 'BP機',
 'B型',
 'B座',
 'B股',
 'B超',
 'B輪',
 'B轮',
 'C#',
 'C++',
 'CALL机',
 'CALL機',
 'CD机',
 'CD機',
 'CD盒',
 'C座',
 'C盘',
 'C盤',
 'C語言',
 'C语言',
 'D座',
 'D版',
 'D盘',
 'D盤',
 'E化',
 'E座',
 'E盘',
 'E盤',
 'E通',
 'F座',
 'F盘',
 'F盤',
 'G盘',
 'G盤',
 'H盘',
 'H盤',
 'H股',
 'IC卡',
 'IP卡',
 'IP地址',
 'IP电话',
 'IP電話',
 'I盘',
 'I盤',
 'K党',
 'K歌之王',
 'K黨',
 'N年',
 'O型',
 'PC机',
 'PC機',
 'PH值',
 'QQ号',
 'QQ號',
 'Q版',
 'RSS訂閱',
 'RSS订阅',
 'SIM卡',
 'T台',
 'T型台',
 'T型臺',
 'T恤',
 'T恤衫',
 'T盘',
 'T盤',
 'T臺',
 'U盘',
 'U盤',
 'VISA卡',
 'X光',
 'X光線',
 'X光线',
 'X射線',
 'X射线',
 'Z盘',
 'Z盤',
 'c#',
 'c++',
 'γ射線',
 'γ射线',
 '䰾',
 '䲁',
 '䴉',
 '一',
 '一一',
 '一一二',
 '一一例',
 '一一分',
 '一一列举',
 '一一列舉',
 '一一对',
 '一一对应',
 '一一對',
 '一一對應',
 '一一記',
 '一一记',
 '一一道來',
 '一一道来',
 '一丁',
 '一丁不識',
 '一丁不识',
 '一丁点',
 '一丁点儿',
 '一丁點',
 '一丁點兒',
 '一七',
 '一七八不',
 '一万',
 '一万一千',
 '一万一千五百二十颗',
 '一万一千八百八十斤',
 '一万一千多间',
 '一万一千零九十五册

In [52]:
len(defaultList)

584429

In [53]:
len(cedictList)

116982

In [49]:
len(set(defaultList).intersection(set(cedictList)))

81195

In [50]:
len(set(defaultList).union(set(cedictList)))

616851

In [2]:
data = pd.read_json("C:\chinadump.json", encoding="utf-8" )

In [3]:
data.head()

,chinadump
0,"{'doc_date': '2019-02-21', 'doc_country': '俄罗斯..."
1,"{'doc_date': '2019-02-21', 'doc_country': '萨摩亚..."
2,"{'doc_date': '2019-02-21', 'doc_country': '巴巴多..."
3,"{'doc_date': '2019-02-21', 'doc_country': '多米尼..."
4,"{'doc_date': '2019-02-21', 'doc_country': '吉尔吉..."


In [4]:
nd = []

for i in data.index:
    d = data.iloc[i,0]
    nd.append(list(d.values()))

In [5]:
data = pd.DataFrame(nd)

In [6]:
data.columns = ['doc_date','doc_country','doc_content']

In [7]:
data.head()

,doc_date,doc_country,doc_content
0,2019-02-21,俄罗斯,\n 为纪念中俄建交70周年，四川省歌舞剧院倾心打造的舞剧《长风啸》于2019年2月19日...
1,2019-02-21,萨摩亚,\n 2019年2月18日，驻萨摩亚大使王雪峰会见援萨摩亚太平洋运动会体育场馆项目中期验收...
2,2019-02-21,巴巴多斯,\n 2019年2月21日，国务委员兼外交部长王毅在北京会见来华参加中国和加勒比建交国外交...
3,2019-02-21,多米尼克,\n\n 2019年2月21日，中国与安提瓜和巴布达、巴哈马、巴巴多斯、多米尼克、格林纳达...
4,2019-02-21,吉尔吉斯斯坦,\n 2019年2月21日，国务委员兼外交部长王毅在北京与吉尔吉斯斯坦外长艾达尔别科夫举行...


In [8]:
data.shape

(132036, 3)

In [45]:
count = CountVectorizer(analyzer="char_wb", ngram_range=(2,5), min_df=2, binary=True)

In [20]:
countries = np.unique(data["doc_country"].copy())

In [10]:
docs = data['doc_content'].copy()

In [11]:
docs[0]

'\n\u3000\u3000为纪念中俄建交70周年，四川省歌舞剧院倾心打造的舞剧《长风啸》于2019年2月19日晚在克里姆林宫剧院隆重上演。驻俄罗斯大使李辉和夫人史晓玲女士观看演出。\n\u3000\u3000该剧运用浪漫神话形式，寓意人类大爱和维护和平主题，受到观众热烈欢迎。演出结束后，李大使夫妇上台慰问全体演职人员，感谢他们在元宵佳节为中俄民众奉献了独具特色的中国舞台艺术产品。\n\n\u3000\u3000李大使表示，此次演出是中国地方文化走出去的有益尝试，为今年中俄两国共同庆祝建交70周年营造了良好氛围，为弘扬中华文化、增进两国人民相互了解发挥了积极作用。希望四川省歌舞剧院深入挖掘当地特色文化资源，创作出更多优秀剧目，继续进入俄罗斯演艺市场，不断提升中国文化在国际舞台的形象和影响力。最后，李大使和史晓玲女士再次向演职人员表示祝贺，并致以节日的问候。\n\n\n\n\n\xa0\n\xa0\n'

In [40]:
docs.shape

(132036,)

In [46]:
countries[:10]

array(['七十七国集团', '上海合作组织', '不丹', '不结盟运动', '世界动物卫生组织', '世界气象组织', '世界经济论坛',
       '东亚峰会（EAS）', '东亚－拉美合作论坛', '东南亚国家联盟（ASEAN）'], dtype=object)

In [47]:
countries.shape

(246,)

In [48]:
cdict = []

In [50]:
for i in countries:
    print(data.loc[data['doc_country']==i,:].shape)
    if data.loc[data['doc_country']==i,:].shape[0] < 2:
        continue
    text = data.loc[data['doc_country']==i,'doc_content'].copy()
    vocab = set(count.fit(text).get_feature_names())
    vocab = vocab.difference(cdict[-1])
    cdict.append(vocab)

(1, 3)
(454, 3)
(26, 3)
(15, 3)
(1, 3)
(1, 3)
(88, 3)
(71, 3)
(167, 3)
(953, 3)
(681, 3)
(81, 3)
(61, 3)
(4, 3)
(2, 3)
(31, 3)
(10, 3)
(27, 3)
(34, 3)
(23, 3)
(812, 3)
(280, 3)
(834, 3)
(838, 3)
(1255, 3)
(655, 3)
(774, 3)
(442, 3)
(704, 3)
(150, 3)
(281, 3)
(131, 3)
(23, 3)
(57, 3)
(552, 3)
(1161, 3)
(389, 3)
(12, 3)
(466, 3)
(394, 3)
(2134, 3)
(533, 3)
(625, 3)
(127, 3)
(286, 3)
(633, 3)
(620, 3)
(9, 3)
(365, 3)
(147, 3)
(171, 3)
(543, 3)
(2, 3)
(375, 3)
(403, 3)
(737, 3)
(466, 3)
(37, 3)
(1, 3)
(3, 3)
(488, 3)
(7, 3)
(8, 3)
(1303, 3)
(582, 3)
(486, 3)
(581, 3)
(215, 3)
(1645, 3)
(719, 3)
(891, 3)
(530, 3)
(402, 3)
(376, 3)
(568, 3)
(231, 3)
(927, 3)
(714, 3)
(495, 3)
(424, 3)
(30, 3)
(272, 3)
(693, 3)
(7, 3)
(36, 3)
(7, 3)
(374, 3)
(1102, 3)
(1243, 3)
(858, 3)
(9, 3)
(811, 3)
(473, 3)
(355, 3)
(597, 3)
(601, 3)
(324, 3)
(1195, 3)
(662, 3)
(396, 3)
(304, 3)
(32, 3)
(136, 3)
(3, 3)
(725, 3)
(364, 3)
(576, 3)
(523, 3)
(632, 3)
(3, 3)
(7, 3)
(273, 3)
(1, 3)
(1167, 3)
(719, 3)
(1271, 3)


KeyboardInterrupt: 

In [ ]:
for i in cdict:
    print(len(i))

In [ ]:
newset = set()

for i in cdict:
    newset = newset.union(i)

In [ ]:
len(newset)